## Preprocessing

In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_application_df = application_df.drop(columns=['EIN','NAME'])
clean_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
unique_values = clean_application_df.nunique()
unique_values.head()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = clean_application_df['APPLICATION_TYPE'].value_counts()
threshold = 3
clean_application_df['APPLICATION_TYPE'] = clean_application_df['APPLICATION_TYPE'].apply(lambda x: x if application_type_counts[x] >= threshold else 'other')
application_type_counts.head()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 3
application_types_to_replace = application_type_counts[application_type_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other        5
T25          3
T14          3
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = clean_application_df['CLASSIFICATION'].value_counts()
threshold = 5
clean_application_df['CLASSIFICATION'] = clean_application_df['CLASSIFICATION'].apply(lambda x: x if classification_counts[x] >= threshold else 'Other')
classification_counts.head()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: count, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = clean_application_df['CLASSIFICATION'].value_counts()
classification_counts_gt_1 = classification_counts[classification_counts > 1]
classification_counts_gt_1.head()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 3
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
df_numeric = pd.get_dummies(application_df, drop_first=True)

In [11]:
# Split our preprocessed data into our features and target arrays
X = df_numeric.drop('IS_SUCCESSFUL', axis=1).values
Y = df_numeric['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dim = X.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units=64, activation='relu', input_dim=input_dim))

# Second hidden layer
nn.add(Dense(units=32, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

c:\Users\swamp\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │     1,259,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,261,377 (4.81 MB)

 Trainable params: 1,261,377 (4.81 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

# For binary classification, y_train should be float or int, depending on the output activation function.
Y_train = np.asarray(Y_train).astype(np.float32)  # or np.int if necessary
Y_test = np.asarray(Y_test).astype(np.float32) 
nn.fit(X_train, Y_train, epochs=100, batch_size=31, validation_split=0.2)


Epoch 1/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - accuracy: 0.5052 - loss: 114071.2188 - val_accuracy: 0.4820 - val_loss: 4566.2676
Epoch 2/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.4996 - loss: 18320.5059 - val_accuracy: 0.5171 - val_loss: 39516.0156
Epoch 3/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.4986 - loss: 14423.6797 - val_accuracy: 0.5186 - val_loss: 6788.8340
Epoch 4/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.5039 - loss: 9664.5625 - val_accuracy: 0.5177 - val_loss: 2668.7058
Epoch 5/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5094 - loss: 3034.5620 - val_accuracy: 0.5228 - val_loss: 1871.0419
Epoch 6/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.5013 - loss: 2208.3528 - val_accuracy: 0.4770 - val_loss: 2912.6389
Epoch 7/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.4978 - loss: 783.9588 - val_accuracy: 0.5180 - val_loss: 27.6746
Epoch 8/100
709/709 ━━━━━━━━━━━━━━━━━━━━ 8s 1

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - 3ms/step - accuracy: 0.5052 - loss: 0.7935
Loss: 0.7935044169425964, Accuracy: 0.5052478313446045


In [22]:
# Export our model to HDF5 file
nn.save('my_model.h5')

Key Findings:
The model was trained over 100 epochs to classify the target variable (IS_SUCCESSFUL).
The final training accuracy was 53.94%, with a validation accuracy of 51.71%, which is only marginally better than random guessing (50%).
The model's loss during training and validation decreased significantly in the earlier epochs but plateaued as training continued, indicating a potential issue with model complexity or data features.
Issues Observed:
High initial loss: The initial model loss was extremely high, indicating possible scaling or feature engineering issues.
Low performance: Even after 100 epochs, the accuracy did not improve significantly from 50%, showing that the model may not be capturing the relevant patterns in the data.


Here’s a breakdown of training and validation accuracy and loss over 100 epochs:

Training and Validation Accuracy:

The training accuracy started at 50.52% and ended at 53.94%.
The validation accuracy remained around 51.71%, showing no significant improvement.
Training and Validation Loss:
Loss values fluctuated and plateaued at around 0.69, suggesting that the model might not be improving further.

The deep learning model trained on this dataset showed modest results:

Training Accuracy: 53.94%
Validation Accuracy: 51.71%
Final Loss: 0.69 for both training and validation sets
These results indicate that the model did not perform significantly better than random guessing (50% accuracy). The deep neural network struggled to learn meaningful patterns from the data, which may suggest issues such as:

Lack of sufficient feature engineering.
The data may not have strong linear separability for classification.
Possible overfitting or underfitting.